In [ ]:
!pip3 install -q datasets peft accelerate evaluate scikit-learn rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 202

In [ ]:
!pip3 install -q --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 79.7 MB/s eta 0:00:00


In [ ]:
!pip3 install -q huggingface_hub

In [ ]:
import torch, torch.nn as nn, random, numpy as np, evaluate
from datasets import load_dataset
from peft import PeftModel
from torch.nn.utils.rnn import pad_sequence

from transformers import GPT2Model
from transformers import (
    GPT2TokenizerFast, GPT2Model, GPT2LMHeadModel,
    TrainingArguments, Trainer
)
from peft import LoraConfig, get_peft_model, TaskType
import evaluate

device = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42
torch.manual_seed(SEED); np.random.seed(SEED); random.seed(SEED)

tok = GPT2TokenizerFast.from_pretrained("gpt2")
tok.pad_token = tok.eos_token

2025-05-06 09:05:27.380312: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746522327.564591      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746522327.616778      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [5]:
# ╔═══════════════════════════════════════════════════════════╗
# 2A.  SENTIMENT HEAD                                         #
# ╚═══════════════════════════════════════════════════════════╝
class GPT2Sentiment(nn.Module):
    def __init__(self, base, n_labels=2):
        super().__init__()
        self.base = base
        self.config = base.config
        self.dropout = nn.Dropout(0.1)
        self.cls = nn.Linear(base.config.hidden_size, n_labels)
    def forward(self, input_ids=None, attention_mask=None,
                labels=None, **kwargs):
        kwargs.pop("num_items_in_batch", None)
        hs = self.base(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        ).last_hidden_state

        logits = self.cls(self.dropout(hs[:, -1]))
        loss = None
        if labels is not None:
            loss = nn.functional.cross_entropy(logits, labels)
        return {"loss": loss, "logits": logits}

base_cls = GPT2Model.from_pretrained("gpt2")
sent_model = GPT2Sentiment(base_cls)
sent_lora = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05,
    task_type=TaskType.SEQ_CLS, target_modules=["c_attn"]
)
sent_model = get_peft_model(sent_model, sent_lora).to(device)

# ╔═══════════════════════════════════════════════════════════╗
# 2B.  SUMMARIZER (LM)                                        #
# ╚═══════════════════════════════════════════════════════════╝
sum_model = GPT2LMHeadModel.from_pretrained("gpt2")
sum_lora = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05,
    task_type=TaskType.CAUSAL_LM, target_modules=["c_attn"]
)
sum_model = get_peft_model(sum_model, sum_lora).to(device)
sum_model.resize_token_embeddings(len(tok))

# ╔═══════════════════════════════════════════════════════════╗
# 2C.  SPAN-QA HEAD                                           #
# ╚═══════════════════════════════════════════════════════════╝
class GPT2SpanQA(nn.Module):
    def __init__(self, base):
        super().__init__()
        self.base = base
        self.config = base.config
        H = base.config.hidden_size
        self.s = nn.Linear(H, 1)
        self.e = nn.Linear(H, 1)
    def forward(self, input_ids=None, attention_mask=None,
                start_positions=None, end_positions=None, labels=None,**kwargs):
        kwargs.pop("num_items_in_batch", None)
        hs = self.base(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        ).last_hidden_state
        start = self.s(hs).squeeze(-1)
        end   = self.e(hs).squeeze(-1)
        loss  = None
        if start_positions is not None:
            ce  = nn.CrossEntropyLoss()
            loss = (ce(start,start_positions)+ce(end,end_positions))/2
        return {"loss": loss, "start_logits": start, "end_logits": end}

base_qa = GPT2Model.from_pretrained("gpt2")
qa_model = GPT2SpanQA(base_qa)
qa_lora = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05,
    task_type=TaskType.TOKEN_CLS, target_modules=["c_attn"]
)
qa_model = get_peft_model(qa_model, qa_lora).to(device)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
# ╔═══════════════════════════════════════════════════════════╗
# 3.  QUICK LoRA FINE-TUNE LOOPS (tiny slices for demo)       #
# ╚═══════════════════════════════════════════════════════════╝
def hr(t): print("\n" + "═"*15 + " " + t + " " + "═"*15)

# ── Sentiment (Amazon Polarity) ──────────────────────────────
hr("Training Sentiment")
raw = load_dataset("amazon_polarity")

def tok_sent(ex):
    return tok(ex["title"] + " " + ex["content"],
               truncation=True, max_length=256)

train_s = raw["train"].shuffle(seed=SEED).select(range(100_000)).map(tok_sent)
val_s   = raw["test"] .select(range(20_000)).map(tok_sent)
train_s = train_s.rename_column("label", "labels")
val_s   = val_s.rename_column("label", "labels")

args_s = TrainingArguments(
    output_dir="sent-lora",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    max_grad_norm=0.1,
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to=[]
)

Trainer(sent_model, args_s,
        train_dataset=train_s,
        eval_dataset=val_s,
        tokenizer=tok).train()
sent_model.save_pretrained("sent-lora")
base = GPT2Model.from_pretrained("gpt2")

sent_adapter = PeftModel.from_pretrained(base, "sent-lora")
sent_adapter.push_to_hub(
    repo_id="Quan0207/GPT-2_finetune",
    adapter_name="sentiment",
    commit_message="Add fine-tuned sentiment LoRA adapter"
)



═══════════════ Training Sentiment ═══════════════


README.md:   0%|          | 0.00/6.81k [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/260M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/258M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/255M [00:00<?, ?B/s]

...

/tmp/ipykernel_19/3374560295.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  Trainer(sent_model, args_s,
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,1.291200,0.180482
2,0.656600,0.157645
3,0.596000,0.158445


/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.h.0.attn.c_attn.lora_A.default.weight', 'base_model.model.h.0.attn.c_attn.lora_B.default.weight', 'base_model.model.h.1.attn.c_attn.lora_A.default.weight', 'base_model.model.h.1.attn.c_attn.lora_B.default.weight', 'base_model.model.h.2.attn.c_attn.lora_A.default.weight', 'base_model.model.h.2.attn.c_attn.lora_B.default.weight', 'base_model.model.h.3.attn.c_attn.lora_A.default.weight', 'base_model.model.h.3.attn.c_attn.lora_B.default.weight', 'base_model.model.h.4.attn.c_attn.lora_A.default.weight', 'base_model.model.h.4.attn.c_attn.lora_B.default.weight', 'base_model.model.h.5.attn.c_attn.lora_A.default.weight', 'base_model.model.h.5.attn.c_attn.lora_B.default.weight', 'base_model.model.h.6.attn.c_attn.lora_A.default.weight', 'base_model.model.h.6.attn.c_attn.lora_B.default.weight', 'base_model.model.h.7.attn.c_attn.lora_A.default.weig

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Quan0207/GPT-2_finetune/commit/8e1b42082c7d27016e1e6620326d2798ebb8313c', commit_message='Add fine-tuned sentiment LoRA adapter', commit_description='', oid='8e1b42082c7d27016e1e6620326d2798ebb8313c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Quan0207/GPT-2_finetune', endpoint='https://huggingface.co', repo_type='model', repo_id='Quan0207/GPT-2_finetune'), pr_revision=None, pr_num=None)

In [7]:
# ── Summarization (CNN/DailyMail) ────────────────────────────
hr("Training Summarizer")
train_raw = load_dataset("cnn_dailymail", "3.0.0", split="train[:100000]")
val_raw   = load_dataset("cnn_dailymail", "3.0.0", split="validation[:10000]")

def pack(ex):
    seq = ex["article"] + tok.eos_token + ex["highlights"] + tok.eos_token
    ids = tok(seq, truncation=True, max_length=1024)["input_ids"]
    return {"input_ids": ids, "labels": ids}

train_sum = train_raw.map(pack, remove_columns=train_raw.column_names)
val_sum   = val_raw.map(pack, remove_columns=val_raw.column_names)

def coll_lm(batch):
    ids = [torch.tensor(x["input_ids"]) for x in batch]
    pad = tok.pad_token_id
    input_ids = torch.nn.utils.rnn.pad_sequence(
        ids, batch_first=True, padding_value=pad
    )
    labels = input_ids.clone()
    return {"input_ids": input_ids, "labels": labels}

args_sum = TrainingArguments(
    output_dir="sum-lora",
    num_train_epochs=5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=5e-5,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    max_grad_norm=0.1,
    fp16=torch.cuda.is_available(),

    logging_strategy="steps",
    logging_steps=200,
    save_strategy="epoch",
    load_best_model_at_end=True,
    eval_strategy="epoch",
    metric_for_best_model="eval_loss",
    report_to=[]
)
trainer_sum = Trainer(
    model=sum_model,
    args=args_sum,
    train_dataset=train_sum,
    eval_dataset=val_sum,
    data_collator=coll_lm,
    tokenizer=tok
)
sum_model.save_pretrained("sum-lora")
lm   = GPT2LMHeadModel.from_pretrained("gpt2")
summ = PeftModel.from_pretrained(lm, "sum-lora")
summ.push_to_hub("Quan0207/GPT-2_finetune", adapter_name="summarization")


═══════════════ Training Summarizer ═══════════════


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/tmp/ipykernel_19/3215215857.py:42: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_sum = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


CommitInfo(commit_url='https://huggingface.co/Quan0207/GPT-2_finetune/commit/a152f459430878593f49fac197f3c3d4c8a98324', commit_message='Upload model', commit_description='', oid='a152f459430878593f49fac197f3c3d4c8a98324', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Quan0207/GPT-2_finetune', endpoint='https://huggingface.co', repo_type='model', repo_id='Quan0207/GPT-2_finetune'), pr_revision=None, pr_num=None)

In [8]:
import tqdm
hr("Evaluation")

# ── 4-A  Sentiment ───────────────────────────────────────────
sent_val = load_dataset("amazon_polarity", split="test[:10000]")
preds, refs = [], []
for ex in tqdm.tqdm(sent_val, desc="Sentiment"):
    enc = tok(ex["title"] + " " + ex["content"],
              return_tensors="pt", truncation=True, max_length=256).to(device)
    logits = sent_model(**enc)["logits"]
    preds.append(int(logits.argmax(-1)))
    refs.append(ex["label"])

acc = evaluate.load("accuracy").compute(predictions=preds, references=refs)["accuracy"]
f1  = evaluate.load("f1").compute(predictions=preds, references=refs)["f1"]
print(f"Sentiment –  Acc: {acc:.4f}   F1: {f1:.4f}")

# ── 4-B  Summarization ───────────────────────────────────────
rouge = evaluate.load("rouge")
val_sum = load_dataset("cnn_dailymail", "3.0.0", split="validation[:5000]")

gens, refs_sum = [], []
for ex in tqdm.tqdm(val_sum, desc="Summaries"):
    inp = tok(ex["article"] + tok.eos_token,
              return_tensors="pt", truncation=True, max_length=900).to(device)
    out_ids = sum_model.generate(**inp, max_new_tokens=80,
                                 pad_token_id=tok.eos_token_id)
    full = tok.decode(out_ids[0], skip_special_tokens=True)
    gens.append(full.split(tok.eos_token)[-1].strip())
    refs_sum.append(ex["highlights"])

print("Summarization ROUGE:",
      rouge.compute(predictions=gens, references=refs_sum, use_stemmer=True))



═══════════════ Evaluation ═══════════════


Sentiment: 100%|██████████| 10000/10000 [02:53<00:00, 57.60it/s]


Sentiment –  Acc: 0.9454   F1: 0.9463


Summaries: 100%|██████████| 5000/5000 [1:16:20<00:00,  1.09it/s]


Summarization ROUGE: {'rouge1': 0.14379530978517396, 'rouge2': 0.07578534481643753, 'rougeL': 0.10168238866310839, 'rougeLsum': 0.1240861654612187}


In [9]:
import evaluate
import torch
from transformers import GPT2Model
from peft import PeftModel
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
from torch.nn.utils.rnn import pad_sequence

# Load the SQuAD dataset
hr("Training QA")
raw_qa = load_dataset("squad", split="train[:80000]")

def tok_qa(ex):
    q, c = ex["question"], ex["context"]
    a = ex["answers"]["text"][0]

    enc = tok(
        q + tok.eos_token + c,
        truncation=True,
        max_length=512,
        return_offsets_mapping=True
    )
    offsets = enc.pop("offset_mapping")
    input_ids = enc["input_ids"]
    attention_mask = enc["attention_mask"]

    idx = c.find(a)
    if idx == -1:
        return {"input_ids": input_ids,
                "attention_mask": attention_mask,
                "start_positions": None,
                "end_positions": None}

    qlen = len(q) + len(tok.eos_token)
    start_char = qlen + idx
    end_char = start_char + len(a)

    start_pos = next(
        (i for i, (s, e) in enumerate(offsets) if s <= start_char < e),
        None
    )
    end_pos = next(
        (i for i, (s, e) in enumerate(offsets) if s < end_char <= e),
        None
    )

    if start_pos is None or end_pos is None:
        return {"input_ids": input_ids,
                "attention_mask": attention_mask,
                "start_positions": None,
                "end_positions": None}

    return {"input_ids": input_ids,
            "attention_mask": attention_mask,
            "start_positions": start_pos,
            "end_positions": end_pos}

# Filter out examples without valid start positions
keep = ["input_ids", "attention_mask", "start_positions", "end_positions"]
train_qa = raw_qa.map(
    tok_qa,
    batched=False,
    remove_columns=[c for c in raw_qa.column_names if c not in keep]
)

train_qa = train_qa.filter(lambda ex: ex["start_positions"] is not None)

# Validation set preparation
val_raw = load_dataset("squad", split="validation[:5000]")
val_qa = val_raw.map(
    tok_qa,
    batched=False,
    remove_columns=val_raw.column_names
).filter(lambda ex: ex["start_positions"] is not None)

# Collate function to handle padding
def coll_qa(batch):
    input_ids = [torch.tensor(ex["input_ids"]) for ex in batch]
    masks = [torch.tensor(ex["attention_mask"]) for ex in batch]
    pad_id = tok.pad_token_id

    input_ids_padded = pad_sequence(
        input_ids, batch_first=True, padding_value=pad_id
    )
    masks_padded = pad_sequence(
        masks, batch_first=True, padding_value=0
    )
    starts = torch.tensor([ex["start_positions"] for ex in batch])
    ends = torch.tensor([ex["end_positions"] for ex in batch])

    return {
        "input_ids": input_ids_padded,
        "attention_mask": masks_padded,
        "start_positions": starts,
        "end_positions": ends,
    }

# Compute the metrics (EM and F1)
metric_squad = evaluate.load("squad")

def compute_metrics_qa(p):
    # Get predictions and labels
    preds = p.predictions
    labels = p.label_ids

    start_pred = preds[0]
    end_pred = preds[1]

    # Convert the start and end predictions to text
    decoded_preds = tok.decode(start_pred, skip_special_tokens=True)

    # Calculate EM and F1
    em = metric_squad.compute(predictions=decoded_preds, references=labels)["exact_match"]
    f1 = metric_squad.compute(predictions=decoded_preds, references=labels)["f1"]

    return {"eval_exact_match": em, "eval_f1": f1}

# TrainingArguments with matching eval & save strategy
args_qa = TrainingArguments(
    output_dir="qa-lora",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    warmup_steps=500,
    lr_scheduler_type="cosine",
    max_grad_norm=0.1,
    fp16=torch.cuda.is_available(),
    logging_strategy="steps",
    logging_steps=250,
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to=[]
)

# Trainer setup with eval and metric computation
trainer_qa = Trainer(
    model=qa_model,
    args=args_qa,
    train_dataset=train_qa,
    eval_dataset=val_qa,          # Evaluation dataset
    data_collator=coll_qa,
    tokenizer=tok,
    compute_metrics=compute_metrics_qa  # Pass the compute_metrics function
)

# Train and save the model
trainer_qa.train()
qa_model.save_pretrained("qa-lora")
PeftModel.from_pretrained(GPT2Model.from_pretrained("gpt2"),
                         "qa-lora"
).push_to_hub(
    "Quan0207/GPT-2_finetune",
    adapter_name="qa"
)



═══════════════ Training QA ═══════════════


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

/tmp/ipykernel_19/3248642362.py:137: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_qa = Trainer(
No label_names provided for model class `PeftModelForTokenClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
0,8.021900,No log
1,7.301900,No log
2,7.000000,No log


/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.h.0.attn.c_attn.lora_A.default.weight', 'base_model.model.h.0.attn.c_attn.lora_B.default.weight', 'base_model.model.h.1.attn.c_attn.lora_A.default.weight', 'base_model.model.h.1.attn.c_attn.lora_B.default.weight', 'base_model.model.h.2.attn.c_attn.lora_A.default.weight', 'base_model.model.h.2.attn.c_attn.lora_B.default.weight', 'base_model.model.h.3.attn.c_attn.lora_A.default.weight', 'base_model.model.h.3.attn.c_attn.lora_B.default.weight', 'base_model.model.h.4.attn.c_attn.lora_A.default.weight', 'base_model.model.h.4.attn.c_attn.lora_B.default.weight', 'base_model.model.h.5.attn.c_attn.lora_A.default.weight', 'base_model.model.h.5.attn.c_attn.lora_B.default.weight', 'base_model.model.h.6.attn.c_attn.lora_A.default.weight', 'base_model.model.h.6.attn.c_attn.lora_B.default.weight', 'base_model.model.h.7.attn.c_attn.lora_A.default.weig

CommitInfo(commit_url='https://huggingface.co/Quan0207/GPT-2_finetune/commit/5c7007fb4baee4634de3439ea2ca56b330da828f', commit_message='Upload model', commit_description='', oid='5c7007fb4baee4634de3439ea2ca56b330da828f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Quan0207/GPT-2_finetune', endpoint='https://huggingface.co', repo_type='model', repo_id='Quan0207/GPT-2_finetune'), pr_revision=None, pr_num=None)

In [10]:
# ╔═══════════════════════════════════════════════════════════╗
# 4.  E V A L U A T I O N                                     #
# ╚═══════════════════════════════════════════════════════════╝

import tqdm
hr("Evaluation")

# ── 4-C  Extractive QA ───────────────────────────────────────
metric_squad = evaluate.load("squad")
val_qa = load_dataset("squad", split="validation[:5000]")

pred_list, ref_list = [], []
for i, ex in enumerate(tqdm.tqdm(val_qa, desc="QA")):
    enc = tok(ex["question"] + tok.eos_token + ex["context"],
              return_tensors="pt", truncation=True, max_length=512).to(device)
    outs = qa_model(**enc)
    s = int(torch.argmax(outs["start_logits"]))
    e = int(torch.argmax(outs["end_logits"]))
    ans = tok.decode(enc["input_ids"][0][s:e+1]).strip()

    pred_list.append({"id": str(i), "prediction_text": ans})
    ref_list.append({"id": str(i),
                     "answers": {"text": ex["answers"]["text"],
                                 "answer_start": []}})

print("QA EM / F1:",
      metric_squad.compute(predictions=pred_list, references=ref_list))



═══════════════ Evaluation ═══════════════


QA: 100%|██████████| 5000/5000 [01:29<00:00, 56.18it/s]


QA EM / F1: {'exact_match': 56.72, 'f1': 66.91447299823427}
